In [12]:
#必要なパッケージのインポート

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import reservoirpy as rpy

from scipy.integrate import solve_ivp
import pandas as pd
from reservoirpy.observables import nrmse, rsquare

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


rpy.verbosity(0)

from reservoirpy.nodes import Reservoir, Ridge
from reservoirpy.datasets import mackey_glass

# just a little tweak to center the plots, nothing to worry about
from IPython.core.display import HTML
HTML("""
<style>
.img-center {
    display: block;
    margin-left: auto;
    margin-right: auto;
    }
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
    }
</style>
""")

rpy.set_seed(42)

%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs


In [13]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# パラメータ
n = 4
vm = 0.505
vd = 1.4
ks = 0.5
k1 = 0.5
k2 = 0.6
Km = 0.5
Kd = 0.13
KI = 1
vs_min = 1.6

# vs_max の値とそれに対応する A の計算
vs_max_values = [1.6, 2.25, 3.1, 4.2]
A_values = [(vs_max / vs_min) - 1 for vs_max in vs_max_values]

# vs を時間 t に対する関数として計算する関数
def vs_function(t, A):
    return vs_min * (1 + A * np.sin(t))

# 微分方程式
def model(t, y, A):
    M, FC, FN = y
    vs = vs_function(t, A)  # vs の値を時間 t に応じて計算
    dMdt = vs * (KI**n / (KI**n + FN**n)) - vm * (M / (Km + M))
    dFCdt = ks * M - vd * (FC / (Kd + FC)) - k1 * FC + k2 * FN
    dFNdt = k1 * FC - k2 * FN
    return [dMdt, dFCdt, dFNdt]

# 初期値と時間設定
y0 = [0.5, 0.5, 0.5]
t_span = [0, 2510]
t_eval = np.linspace(t_span[0], t_span[1], 25100)

# 最初の vs_max の値についてのみシミュレーションとプロット
i = 2
A = A_values[i]
sol = solve_ivp(model, t_span, y0, t_eval=t_eval, args=(A,), max_step=0.01)

In [14]:
# CSVファイルにデータを保存するためのDataFrameを作成
vs_values = vs_function(sol.t, A)  # 外力Ap(t)としてvsを計算
data_frame = pd.DataFrame({
    'time': sol.t,
    'M': sol.y[0],
    'FC': sol.y[1],
    'FN': sol.y[2],
    'vs': vs_values  # 外力Ap(t)としての列
})

# CSVファイルにデータを保存
filename = 'LD_cycle1.1.csv'
data_frame.to_csv(filename, index=False)

# CSVファイルを読み込む
data_loaded = pd.read_csv(filename)

# CSVから値を抽出してNumpy配列に格納
X = data_loaded[['M', 'FC', 'vs']].values  

In [15]:
X.shape

(25100, 3)

In [16]:
from hyperopt import hp, tpe, Trials, fmin

In [17]:
# Objective functions accepted by ReservoirPy must respect some conventions:
#  - dataset and config arguments are mandatory, like the empty '*' expression.
#  - all parameters that will be used during the search must be placed after the *.
#  - the function must return a dict with at least a 'loss' key containing the result
# of the loss function. You can add any additional metrics or information with other 
# keys in the dict. See hyperopt documentation for more informations.
def objective(dataset, config, *, iss, N, sr, lr, ridge, seed):
    
    # This step may vary depending on what you put inside 'dataset'
    train_data, validation_data = dataset
    X_train, y_train = train_data
    X_val, y_val = validation_data
    
    # You can access anything you put in the config 
    # file from the 'config' parameter.
    instances = config["instances_per_trial"]
    
    # The seed should be changed across the instances, 
    # to be sure there is no bias in the results 
    # due to initialization.
    variable_seed = seed 
    
    losses = []; r2s = [];
    for n in range(instances):
        # Build your model given the input parameters
        reservoir = Reservoir(N, 
                              sr=sr, 
                              lr=lr, 
                              input_scaling=iss, 
                              seed=variable_seed)
        
        readout = Ridge(ridge=ridge)

        model = reservoir >> readout


        # Train your model and test your model.
        prediction = model.fit(X_train, y_train) \
                           .run(X_test)
        
        loss = nrmse(y_test, prediction, norm_value=np.ptp(X_train))
        r2 = rsquare(y_test, prediction)
        
        # Change the seed between instances
        variable_seed += 1
        
        losses.append(loss)
        r2s.append(r2)

    # Return a dictionnary of metrics. The 'loss' key is mandatory when
    # using hyperopt.
    return {'loss': np.mean(losses),
            'r2': np.mean(r2s)}

In [18]:
hyperopt_config = {
    "exp": f"hyperopt-LD_cycle1.1", # the experimentation name
    "hp_max_evals": 300,             # the number of differents sets of parameters hyperopt has to try
    "hp_method": "tpe",           # the method used by hyperopt to chose those sets (see below)
    "seed": 42,                      # the random state seed, to ensure reproducibility
    "instances_per_trial": 3,        # how many random ESN will be tried with each sets of parameters
    "hp_space": {                    # what are the ranges of parameters explored
        "N": ["choice", 500],             # the number of neurons is fixed to 500
        "sr": ["loguniform", 1e-2, 10],   # the spectral radius is log-uniformly distributed between 1e-2 and 10
        "lr": ["loguniform", 1e-3, 1],  # idem with the leaking rate, from 1e-3 to 1
        "iss": ["uniform", 0, 1],           # the input scaling uniformly distributed between 0 and 1
        "ridge": ["loguniform", 1e-9, 1e-2],        # and so is the regularization parameter.
        "seed": ["choice", 5555]          # an other random seed for the ESN initialization
    }
}


import json

# we precautionously save the configuration in a JSON file
# each file will begin with a number corresponding to the current experimentation run number.
with open(f"{hyperopt_config['exp']}.config.json", "w+") as f:
    json.dump(hyperopt_config, f)


In [19]:
from reservoirpy.datasets import to_forecasting

train_len = 10000
test_len = 10000

x, y = to_forecasting(X, forecast=1)
X_train, y_train = x[:train_len], y[:train_len]
X_test, y_test = x[train_len:train_len+test_len], y[train_len:train_len+test_len]

dataset = ((X_train, y_train), (X_test, y_test))

In [20]:
from reservoirpy.hyper import research

best = research(objective, dataset, f"{hyperopt_config['exp']}.config.json", ".")

  8%|▊         | 25/300 [01:57<21:39,  4.72s/trial, best loss: 8.208373576580987e-05] 

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=8.67402e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=8.64729e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=8.65963e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



  9%|▉         | 27/300 [02:07<21:33,  4.74s/trial, best loss: 5.694398791668125e-05]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.17636e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=9.8147e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=7.63794e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



 11%|█▏        | 34/300 [02:41<21:38,  4.88s/trial, best loss: 5.694398791668125e-05]

...

 86%|████████▌ | 257/300 [19:53<03:20,  4.66s/trial, best loss: 2.764928973212182e-06]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=9.69913e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=9.32761e-17): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



 96%|█████████▋| 289/300 [22:21<00:50,  4.61s/trial, best loss: 1.0529237911892798e-06]

/Users/manqueenmannequin/miniforge3/envs/python38gen2/lib/python3.8/site-packages/reservoirpy/nodes/readouts/ridge.py:17: LinAlgWarning: Ill-conditioned matrix (rcond=1.01868e-16): result may not be accurate.
  return linalg.solve(XXT + ridge, YXT.T, assume_a="sym")



100%|██████████| 300/300 [23:12<00:00,  4.64s/trial, best loss: 1.0529237911892798e-06]


In [21]:
best

({'N': 0,
  'iss': 0.08953586004134119,
  'lr': 0.9765528770413411,
  'ridge': 4.665293050641528e-09,
  'seed': 0,
  'sr': 0.6321052521221111},
 <hyperopt.base.Trials at 0x107c83c40>)

In [22]:
# `best`タプルの最初の要素には最適化されたハイパーパラメータが直接含まれています
best_params = best[0]

# numpy int64型をPythonのint型に変換するための関数
def convert(o):
    if isinstance(o, np.int64): return int(o)
    raise TypeError

# 最適なハイパーパラメータをJSONファイルに保存
with open(f"{hyperopt_config['exp']}_best_params.json", 'w') as f:
    json.dump(best_params, f, default=convert)